# Búsquedas semánticas con películas

## Entorno:

```shellscript
docker run -p 6333:6333 -p 6334:6334 -v $(pwd)/q_storage:/qdrant/storage:z qdrant/qdrant
```

```shellscript	
python -m venv .venv
source .venv/bin/activate
```

Dataset: https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots


In [9]:
%pip install -r requirements.txt
#!jupyter nbextension enable --py widgetsnbextension


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_json('datasets/star_wars_plots.json')
print('Columns:', df.columns, ', Records:', df.shape[0])
df.head()
startwars_movies = df

Columns: Index(['title', 'plot'], dtype='object') , Records: 11


### Creamos embeddings del argumento de las películas

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-minilm-l6-v2', device='cuda')
#model = SentenceTransformer('all-mpnet-base-v2', device='cuda')
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)


### Preparamos los datos para almacenarlos en la BBDD Vectorial

In [20]:
import time
from tqdm.notebook import tqdm

CHUNK_WORDS = 200

# Creamos chunks de texto de unas 170 palabras con un solape de 10 palabras
def chunk_text(text, chunk_size=CHUNK_WORDS, overlap=CHUNK_WORDS//10):
    '''
    Divide el texto en chunks de un tamaño dado con un solape entre ellos.
    Cada modelo solo puede procesar un número limitado de tokens, por lo que 
    es necesario dividir el texto en chunks.
    1000 tokens son aproximadamente 750 palabras.
    https://openai.com/pricing#language-models
    '''
    chunks = []
    words = text.split()
    for i in range(0, len(words), chunk_size-overlap):
        chunks.append(' '.join(words[i:i+chunk_size]))
    return chunks

# Añadimos columnas plot_chunks y chunk_embeddings que serán arrays en  startwars_movies
plot_embeddings = []
for i, row in tqdm(startwars_movies.iterrows()):
    plot_chunks = chunk_text(row['plot'])
    embeddings = model.encode(plot_chunks).tolist()
    plot_embeddings.append([{'chunk': chunk, 'embedding': embedding} for chunk, embedding in zip(plot_chunks, embeddings)])
    print(i, row['title'], 'chunks:', len(plot_chunks))
    
#startwars_movies['embedding'] = model.encode(startwars_movies['plot'].tolist(), show_progress_bar=True).tolist()
#startwars_movies.dropna(inplace=True)
startwars_movies.head()


0it [00:00, ?it/s]

0 Star Wars: Episodio IV - Una nueva esperanza chunks: 8
1 Star Wars: Episodio V - El Imperio contraataca chunks: 12
2 Star Wars: Episode VI - Return of the Jedi chunks: 18
3 Star Wars: Episodio I - La amenaza fantasma chunks: 6
4 Star Wars: Episodio II - El ataque de los clones chunks: 14
5 Star Wars: Episodio III - La venganza de los Sith chunks: 29
6 Star Wars: Episodio VII - El despertar de la Fuerza chunks: 21
7 Star Wars: Episodio VIII - Los últimos Jedi chunks: 17
8 Star Wars: Episodio IX - El ascenso de Skywalker chunks: 18
9 Rogue One: una historia de Star Wars chunks: 21
10 Han Solo: una historia de Star Wars chunks: 6


,title,plot,metadata
0,Star Wars: Episodio IV - Una nueva esperanza,"Hace mucho tiempo en una galaxia muy, muy leja...",{'title': 'Star Wars: Episodio IV - Una nueva ...
1,Star Wars: Episodio V - El Imperio contraataca,"Hace mucho tiempo en una galaxia muy, muy leja...",{'title': 'Star Wars: Episodio V - El Imperio ...
2,Star Wars: Episode VI - Return of the Jedi,"Hace mucho tiempo en una galaxia muy, muy leja...",{'title': 'Star Wars: Episode VI - Return of t...
3,Star Wars: Episodio I - La amenaza fantasma,"Hace mucho tiempo en una galaxia muy, muy leja...",{'title': 'Star Wars: Episodio I - La amenaza ...
4,Star Wars: Episodio II - El ataque de los clones,"Hace mucho tiempo, en una galaxia muy, muy lej...",{'title': 'Star Wars: Episodio II - El ataque ...


In [21]:
movies_metadata = startwars_movies[['title', 'plot']].to_dict(orient='records')
startwars_movies['metadata'] = movies_metadata
startwars_movies.head()

,title,plot,metadata
0,Star Wars: Episodio IV - Una nueva esperanza,"Hace mucho tiempo en una galaxia muy, muy leja...",{'title': 'Star Wars: Episodio IV - Una nueva ...
1,Star Wars: Episodio V - El Imperio contraataca,"Hace mucho tiempo en una galaxia muy, muy leja...",{'title': 'Star Wars: Episodio V - El Imperio ...
2,Star Wars: Episode VI - Return of the Jedi,"Hace mucho tiempo en una galaxia muy, muy leja...",{'title': 'Star Wars: Episode VI - Return of t...
3,Star Wars: Episodio I - La amenaza fantasma,"Hace mucho tiempo en una galaxia muy, muy leja...",{'title': 'Star Wars: Episodio I - La amenaza ...
4,Star Wars: Episodio II - El ataque de los clones,"Hace mucho tiempo, en una galaxia muy, muy lej...",{'title': 'Star Wars: Episodio II - El ataque ...


In [22]:
vector_data = []
i = 0
for k, md in enumerate(startwars_movies['metadata'].tolist()):
    #print(md)
    for plot_chunk in plot_embeddings[k]:
        metadata = dict(title=md['title'])
        i += 1
        metadata['plot_chunk'] = plot_chunk['chunk'] 
        vector_data.append({
            "id": i,
            "embedding": plot_chunk['embedding'],
            "metadata": metadata,
        })

for v in vector_data[:6]:
    print(v['id'], v['metadata']['title'],'-', v['metadata']['plot_chunk'][:80])
print('Vectors:', len(vector_data), 'Movies:', len(startwars_movies))  


1 Star Wars: Episodio IV - Una nueva esperanza - Hace mucho tiempo en una galaxia muy, muy lejana ... Son tiempos de guerra civil
2 Star Wars: Episodio IV - Una nueva esperanza - las tropas imperiales toman el mando de la nave, con Darth Vader a la cabeza. Du
3 Star Wars: Episodio IV - Una nueva esperanza - droide se escapa de la granja de Owen, por lo que Luke y C-3PO acuden en su búsq
4 Star Wars: Episodio IV - Una nueva esperanza - enseñará todo sobre la Fuerza, pero Luke, inicialmente inseguro, rechaza la ofer
5 Star Wars: Episodio IV - Una nueva esperanza - considerable de control mental y no cede ante los interrogatorios, por lo que él
6 Star Wars: Episodio IV - Una nueva esperanza - espacio exterior y en cambio se percatan de la cercanía de la Estrella de la Mue
Vectors: 170 Movies: 11


### Conectamos con Qdrant para crear una colección y alimentarla con los embeddings


In [23]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

qd = QdrantClient(url="http://localhost:6333")

MOVIES_COLLECTION = "movies_sw"

qd.delete_collection(MOVIES_COLLECTION)
# "size" is the dimension of the vectors, "distance" is the metric used to calculate the distance between vectors
qd.create_collection(MOVIES_COLLECTION, VectorParams(size=model.get_sentence_embedding_dimension(), distance=Distance.COSINE))
movies = qd.get_collection(MOVIES_COLLECTION)

movies

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=0, indexed_vectors_count=0, points_count=0, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [24]:
from qdrant_client.models import PointStruct

points = list(map(lambda x: PointStruct(id=int(x['id']), vector=x['embedding'], payload=x['metadata']), vector_data))
print('Points:', len(points), 'Movies:', len(startwars_movies))
op = qd.upsert(
    collection_name=MOVIES_COLLECTION,
    wait=True,
    points=points,
)

print(qd.get_collection(MOVIES_COLLECTION))



Points: 170 Movies: 11
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=170 indexed_vectors_count=0 points_count=170 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}


### Realizamos búsquedas semánticas sobre la saga Star Wars (Spoilers warning!!)

In [29]:


queries = [
    'Anakin gana una carrera cuando era un niño',
    'Los clones reciben la orden de ejecutar a los Jedi',
    'Han Solo gana el Halcón Milenario en una partida de cartas',
    'Kylo Ren mata a su padre, Han Solo',
    'Palpatine es derrotado definitivamente por Rey',
    # Less accurate queries
    'Luke descubre que Darth Vader es su padre',
    'Luke encuentra a Yoda y es entrenado como Jedi',
]

print(plot_embeddings[-1])
for q in queries:
    query_emb = model.encode(q)
    results = qd.search(MOVIES_COLLECTION, query_vector=query_emb, limit=3)
    print('Q:', q)
    for r in results:
        print('     ', r.score, ' => ', r.payload['title'])
        print('     ', f'(id: {r.id})', r.payload['plot_chunk'])



[{'chunk': "La galaxia se encuentra en un estado de desorden, con sindicatos del crimen compitiendo por recursos valiosos como el coaxium, un raro híper combustible que ayuda a cualquier nave a navegar a través del hiperespacio. En el planeta/astillero de Corellia, a los niños huérfanos se les hace robar para sobrevivir, y un joven Han y su amante Qi'ra desean escapar de las garras de una banda criminal local. Con éxito sobornan a un oficial imperial con coaxium robado a cambio de un pasaje en un transporte saliente, pero Qi'ra es aprehendida por sus perseguidores antes de que pueda abordar. Han se compromete a regresar por ella y se une a la Armada Imperial como cadete de vuelo.Tres años más tarde, Han fue expulsado de la Academia Imperial de Vuelo por insubordinación. Mientras se desempeñaba como soldado de infantería durante una batalla en el planeta Mimban, se encuentra con una banda de delincuentes que se hacen pasar por soldados imperiales liderados por Tobias Beckett. Intenta ch